**<h1>Note : The small functions are created in the Part1 which will be used in the part 2 ML Model building. Please don't consider these small function as they are not nesseccery</h1>**



In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from scipy.stats import zscore
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import pickle

%matplotlib inline

drive.mount('/content/drive')

MessageError: ignored

#Part 1

##Part 1.1

###Part 1- 1 a. Read ‘TelcomCustomer-Churn_1.csv’ as a DataFrame and assign it to a variable

In [ ]:
def loadDataSet(filePath) : 
  data = pd.read_csv(filePath)
  print('Shape of the dataset : ',data.shape)
  return data

In [ ]:
churn1 = loadDataSet('/content/drive/MyDrive/Colab Notebooks/Ensemble Techniques Project/TelcomCustomer-Churn_1.csv')
churn1.head()

###Part 1- 1 b.Read ‘TelcomCustomer-Churn_2.csv’ as a DataFrame and assign it to a variable

In [ ]:
churn2 = loadDataSet('/content/drive/MyDrive/Colab Notebooks/Ensemble Techniques Project/TelcomCustomer-Churn_2.csv')
churn2.head()

###Part 1- 1 c.Merge both the DataFrames on key ‘customerID’ to form a single DataFrame

In [ ]:
df = pd.merge(churn1, churn2, on="customerID")
print(df.shape)
df.head()

###Part 1- 1 d.Verify if all the columns are incorporated in the merged DataFrame by using simple comparisonOperator in Python


In [ ]:
def checkIfColumnExist(source,destination): # Function takes the source and destination dataframe and check if all the column from source is present in the destination
  for column in source.columns:
    if not column in destination :
      print('Column {} is not present in the merged data frame'.format(column))
      return
  
  print('All the columns from source are present in the merged data frame')


In [ ]:
checkIfColumnExist(churn1,df)
checkIfColumnExist(churn2,df)

##Part 1.2

###Part 1- 2 a Impute missing/unexpected values in the DataFrame

In [ ]:
def removeUnwantedColumn(df) : 
  df = df.drop(['customerID'],axis=1) # droping the customer id since it is not required
  return df

df = removeUnwantedColumn(df)
df.isna().sum()

The above results shows that there is no null or empty value present in the data

In [ ]:
for column in df.columns:
  print('------------------Value count for {}---------------'.format(column))
  print(df[column].value_counts())

From the above output we can observ that all the categorical value has the valid data

###part 1- 2 b Make sure all the variables with continuous values are of ‘Float’ type


In [ ]:
df.info()

1. From the above output we can see that the 'TotalCharges' column is a object data which means that the column contains invalid data
2. SeniorCitizen is the int data type but it should be changed to categorical variable
3. Churn (Target variable) is in the categorical variable covert to interger

In [ ]:
def converCategoricalToContinues(df) :
  df['SeniorCitizen'] = df['SeniorCitizen'].replace(1, 'Yes')
  df['SeniorCitizen'] = df['SeniorCitizen'].replace(0, 'No')

  df['SeniorCitizen']

  df.TotalCharges[df.TotalCharges.str.isspace()==True]
  df['TotalCharges'] = df[['TotalCharges']].apply(lambda x: x.str.strip()).replace('', 0)
  df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], downcast="float")

  df['Churn'] = df['Churn'].replace('Yes',1)
  df['Churn'] = df['Churn'].replace('No',0)

  return df

df = converCategoricalToContinues(df)
df.info()

1. Successfully converted the 'TotalCharges' column to float data type
2. Successfully converted 'SeniorCitizen' column from int to float with proper value replacement
3. Successfully converted 'Churn' column from object to int

###Part 1- 2 c Create a function that will accept a DataFrame as input and return pie-charts for all the appropriate Categorical features. Clearly show percentage distribution in the pie-chart.


In [ ]:
def createPieChart(df): 
  """
  This method is used to plot the pie chart for all the categorical variable
  """
  df1 = df.select_dtypes(include='object')
  i=1;
  col=3
  row=len(df1.columns)
  plt.figure(figsize=(20,40))

  for column in df1:
    x= df1[column].value_counts();
    plt.subplot(row, col, i)
    plt.title('{} Distribution'.format(column));
    plt.pie(x,labels=x.keys(),autopct= (lambda x : '{:.1f}%'.format(x)));
    i+=1

In [ ]:
createPieChart(df)

###Part 1- 2 d.Share insights for Q2.c 


1. In the given sample the gender and the distribution is split in equally half
2. 70% of the customers are Independent customers
3. Most of the customers are opting for the optical fiber connection which followed by the DSL
4. Cutomer opting for the StreamingTV/Streaming Movies and not opting for the StreamingTV/Streaming Movies is equally discributed (38% to 39% )
5. Customers are more convinient with the paperless billing compare the the physical billing
6. The Highest opted payment method is Electronic main with 33.6% and followd by mailed check and creadit card(auto). Least opted payment method is Bank Transfer (auto)
7. Customer not having internet service will not get the following services<br>
    a. Online Security <br>
    b. Online Backup <br>
    c. Device protection <br>
    d. Tech support <br>
    e. Streaming Tv <br>
    f. Streaming Movies
  

###Part 1 - 2 e.Encode all the appropriate Categorical features with the best suitable approach


In [ ]:
df.columns

In [ ]:
def dataEncoding(df) : 
  oneHotCols=['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
  result= pd.get_dummies(df,columns=oneHotCols)
  return result

result = dataEncoding(df) 
result.head(10)

###Part 1- 2 f.Split the data into 80% train and 20% test. 


In [ ]:
def splitData(df,test_size) : 
  x = df.drop(['Churn'],axis=1)
  y = df['Churn']
  x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=test_size,random_state=1)
  return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test = splitData(result,0.2)

###Part 1 - 2 g.Normalize/Standardize the data with the best suitable approach

In [ ]:
df.describe()

In [ ]:
def standardizingData(x_train,x_test) :
  x_train_scaled=x_train.apply(zscore) #Standardizing the training data
  x_test_scaled=x_test.apply(zscore) #Standardizing the test data. 
  return x_train_scaled,x_test_scaled

x_train_scaled,x_test_scaled = standardizingData(x_train,x_test)

Standardizing the train and test data seperatly since the st and mean changes after the split

##Part 1.3

###Part 1 - 3 a. Train a model using XGBoost. Also print best performing parameters along with train and test performance

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}

# Initialize XGBoost model
xgb_model = xgb.XGBClassifier()

# Define scoring metric
scorer = make_scorer(accuracy_score)

# Perform grid search with cross-validation
grid = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=scorer, cv=5)
grid.fit(x_train_scaled, y_train)

# Print best performing parameters
print("Best performing parameters:", grid.best_params_)

# Predict on training and test set with best performing model
y_train_pred = grid.predict(x_train_scaled)
y_test_pred = grid.predict(x_test_scaled)

# Evaluate model performance on training and test set
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Train accuracy: %.2f%%" % (train_accuracy * 100))
print("Test accuracy: %.2f%%" % (test_accuracy * 100))

###Part 1 - 3 b Improve performance of the XGBoost as much as possible. Also print best performing parameters along with train and test performance.

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'reg_alpha': [0, 0.001, 0.01],
    'reg_lambda': [0, 0.001, 0.01]
}

xgb_model = xgb.XGBClassifier()
scorer = make_scorer(accuracy_score)

# Perform grid search with cross-validation
grid = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=scorer, cv=5)
grid.fit(x_train_scaled, y_train)

# Print best performing parameters
print("Best performing parameters:", grid.best_params_)

# Predict on training and test set with best performing model
y_train_pred = grid.predict(x_train_scaled)
y_test_pred = grid.predict(x_test_scaled)

# Evaluate model performance on training and test set
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Train accuracy: %.2f%%" % (train_accuracy * 100))
print("Test accuracy: %.2f%%" % (test_accuracy * 100))

#Part 2

###Part 2 - Build a machine learning workflow that will run autonomously with the csv file and return best performing model

In [ ]:
def findTheBestPerformingModel(filePath):
  """
  This function will take the CSV file as the input and return the best performing
  Model. This function will print the nesseccery logs and and write the pickel file 
  for the best performing model
  """
  sample_data = loadDataSet(filePath)
  
  print('--------------------------------------------------------------------')
  print('Sample Data : ',sample_data.columns)
  
  # Removing the unwanted data
  sample_data = removeUnwantedColumn(sample_data)

  #Spliting the data into X and Y and also spliting into train and test data
  X_train,X_test,y_train,y_test = splitData(result,0.2)  

  print('--------------------------------------------------------------------')
  print("Independent Variables : ",X_train.columns)
  print('--------------------------------------------------------------------')
  print("Target Variables : ",y_train.name)

  preprocessing_pipeline = Pipeline([
      ('standard_scaler', StandardScaler())
  ])

  rf_param_grid = {
      'rf__n_estimators': [50, 100, 200],
      'rf__max_depth': [3, 5, 7],
      'rf__min_samples_split': [2, 4, 6]
  }

  gb_param_grid = {
      'gb__n_estimators': [50, 100, 200],
      'gb__max_depth': [3, 5, 7],
      'gb__min_samples_split': [2, 4, 6],
      'gb__learning_rate': [0.01, 0.1, 0.5]
  }

  xgb_param_grid = {
      'xgb__n_estimators': [50, 100, 200],
      'xgb__max_depth': [3, 5, 7],
      'xgb__learning_rate': [0.01, 0.1, 0.5],
      'xgb__subsample': [0.5, 0.7, 1.0],
      'xgb__colsample_bytree': [0.5, 0.7, 1.0],
      'xgb__reg_alpha': [0, 0.001, 0.01],
      'xgb__reg_lambda': [0, 0.001, 0.01]
  }

  models = [
      ("rf", RandomForestClassifier()),
      ("gb", GradientBoostingClassifier()),
      ("xgb", XGBClassifier())
  ]

  best_model = None
  best_model_name = ''
  best_score = 0

  for model_name, model in models:

      print('--------------------------------------------------------------------')
      print('Building model : ', model_name)
      # Define full pipeline with preprocessing and model
      full_pipeline = Pipeline([
          ('preprocessing', preprocessing_pipeline),
          (model_name, model)
      ])

      # Define parameter grid for current model
      if model_name == "rf":
          param_grid = rf_param_grid
      elif model_name == "gb":
          param_grid = gb_param_grid
      elif model_name == "xgb":
          param_grid = xgb_param_grid

      grid = GridSearchCV(full_pipeline, param_grid, cv=5, n_jobs=-1)
      grid.fit(X_train, y_train)

      # Print best performing parameters
      print("Best performing parameters for %s: " % model_name)
      print(grid.best_params_)

      # Evaluate performance on test set
      y_pred = grid.predict(X_test)
      score = accuracy_score(y_test, y_pred)
      print("Accuracy score on test set for \'{}\': {:.2f}%" .format (model_name, score*100))

      # Update best model if current model has higher score
      if score > best_score:
        best_model = model
        best_model_name = model_name

      
  # Open a file for writing in binary mode
  pickel_file_name = 'model.pkl'
  with open(pickel_file_name, 'wb') as f:
  # Serialize the data and write it to the file
    pickle.dump(best_model, f)

  print('--------------------------------------------------------------------')
  print('The Best model is : ',best_model_name)  
  print('The pickel file is stored as : ',pickel_file_name)
  print('--------------------------------------------------------------------')

  return best_model;

In [ ]:
filePath = "/content/drive/MyDrive/Colab Notebooks/Ensemble Techniques Project/TelcomCustomer-Churn_2.csv";
findTheBestPerformingModel(filePath)